## Hi This is my data preparation worksheet

In [86]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
      raw_text = f.read()

print("Total number of characters: ", len(raw_text))
raw_text[:99]

Total number of characters:  20479


'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no '

In [31]:
import re

In [32]:
text = "Hello I am just trying this out!"
result = re.split(r'(\s)', text)
print(result)

['Hello', ' ', 'I', ' ', 'am', ' ', 'just', ' ', 'trying', ' ', 'this', ' ', 'out!']


In [33]:
result = re.split(r'[,.]|(\s)', text)
print(result)

['Hello', ' ', 'I', ' ', 'am', ' ', 'just', ' ', 'trying', ' ', 'this', ' ', 'out!']


In [34]:
result = [item for item in result if item.strip()]
result

['Hello', 'I', 'am', 'just', 'trying', 'this', 'out!']

This is a simple example of how a tokenizer works

In [35]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)

In [36]:
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [37]:
print(len(preprocessed))

4690


Convert Tokens into Token IDs

In [38]:
allwords = sorted(set(preprocessed))

In [39]:
vocab_size = len(allwords)
vocab_size

1130

In [40]:
vocab = {token:integer for integer,token in enumerate(allwords)}

In [41]:
for i,item in enumerate(vocab.items()):
    print(i,"->",item)
    if i>=20:
        break

0 -> ('!', 0)
1 -> ('"', 1)
2 -> ("'", 2)
3 -> ('(', 3)
4 -> (')', 4)
5 -> (',', 5)
6 -> ('--', 6)
7 -> ('.', 7)
8 -> (':', 8)
9 -> (';', 9)
10 -> ('?', 10)
11 -> ('A', 11)
12 -> ('Ah', 12)
13 -> ('Among', 13)
14 -> ('And', 14)
15 -> ('Are', 15)
16 -> ('Arrt', 16)
17 -> ('As', 17)
18 -> ('At', 18)
19 -> ('Be', 19)
20 -> ('Begin', 20)


In [42]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1',text)
        return text

In [43]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [44]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [45]:
sampleText = "Hello, do you like tea?"
print(tokenizer.encode(sampleText))

KeyError: 'Hello'

Adding Special Context tokens

In [46]:
alltokens = sorted(set(preprocessed))
alltokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(alltokens)}

In [47]:
len(vocab.items())

1132

In [50]:
class SimpleTokenizerV2 :
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed [
            item if item in self.str_to_int
            else [ "<|unk|>" for item in preprocessed ]
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1',text)
        return text

In [52]:
tokenizer = SimpleTokenizerV2
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [53]:
tokenizer.encode(text)

TypeError: SimpleTokenizerV2.encode() missing 1 required positional argument: 'text'

## Byte Pair Encoding (BPE)

Used in GPT-2 and GPT-3

In [54]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 894.9/894.9 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [56]:
tokenizer = tiktoken.get_encoding("gpt2")

In [57]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
        "of someunknownPlace.")

In [58]:
integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [59]:
 strings = tokenizer.decode(integers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

### Create Input - Target data pairs

In [60]:
enc_text = tokenizer.encode(raw_text)

In [62]:
print(len(enc_text))

5145


context size is the size of tokens included in the input

In [67]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [70]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "--->", desired)
    print(tokenizer.decode(context), "--->", tokenizer.decode([desired]))

[290] ---> 4920
 and --->  established
[290, 4920] ---> 2241
 and established --->  himself
[290, 4920, 2241] ---> 287
 and established himself --->  in
[290, 4920, 2241, 287] ---> 257
 and established himself in --->  a


But we need these in the form of tensors as pytorch always works with tensors (i/p tensor & o/p tensor)

#### Implementing a Data Loader

In [71]:
from torch.utils.data import DataLoader, Dataset

In [78]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.output_ids = []
        token_ids = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})
        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.output_ids[idx]
        

In [79]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last = True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

Difference between batch_size and num_workers => batch size is the number of operations the model perform before updating its parameters, while num_workers is the parallel procession consumption of cpus/gpus

In [80]:
import torch

In [81]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_itr = iter(dataloader)
first_batch = next(data_itr)
print(first_batch)

[tensor([[15496,    11,   466,   345]]), tensor([[ 11, 466, 345, 588]])]


In [82]:
second_batch = next(data_itr)

In [83]:
second_batch

[tensor([[ 11, 466, 345, 588]]), tensor([[ 466,  345,  588, 8887]])]

In [89]:
dataloader2 = create_dataloader_v1(raw_text, batch_size=4, max_length=4, stride=4, shuffle=False)

data_itr2 = iter(dataloader2)
inputs,targets = next(data_itr2)
print(inputs, "--->", targets)

tensor([[15496,    11,   466,   345],
        [  588,  8887,    30,   220],
        [50256,   554,   262,  4252],
        [18250,  8812,  2114,  1659]]) ---> tensor([[   11,   466,   345,   588],
        [ 8887,    30,   220, 50256],
        [  554,   262,  4252, 18250],
        [ 8812,  2114,  1659,   617]])


In [1]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   - -------------------------------------- 0.8/24.0 MB 6.7 MB/s eta 0:00:04
   -- ------------------------------------- 1.6/24.0 MB 3.6 MB/s eta 0:00:07
   --- ------------------------------------ 2.1/24.0 MB 3.2 MB/s eta 0:00:07
   ----- ---------------------------------- 3.1/24.0 MB 3.9 MB/s eta 0:00:06
   ------ --------------------------------- 4.2/24.0 MB 3.9 MB/s eta 0:00:06
   -------- ------------------------------- 5.0/24.0 MB 3.9 MB/s eta 0:00:05
   --------- ------------------------------ 5.8/24.0 MB 3.9 MB/s eta 0:00:05
   ---------- ----------------------------- 6.6/24.0 MB 3.9 MB/s eta 0:00:05
   ------------ --------------------------- 7.3/24.0 MB 3.9 MB/s eta 0:00:05
   ------------- -------------------------- 8.1/24.0 MB 3.9 MB/s eta 0:00:05
   -------------- ------------------------- 8.9/24.0 MB 3.9 MB/s eta 0:00:04
   --


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import gensim.downloader as api

In [2]:
model = api.load("word2vec-google-news-300")

300 in this stands for 300 dim vector

In [3]:
word_vectors = model
print(word_vectors['computer'])

[ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01
  5.51757812e-02  5.02929

In [5]:
print(word_vectors['cat'].shape)

(300,)


King + Women - Man = ?

In [6]:
print(word_vectors.most_similar(positive=['king', 'women'], negative=['man'], topn=10))

[('queen', 0.4827326238155365), ('queens', 0.466781347990036), ('kumaris', 0.4653734564781189), ('kings', 0.4558638632297516), ('womens', 0.422832190990448), ('princes', 0.4176960587501526), ('Al_Anqari', 0.41725507378578186), ('concubines', 0.4011078476905823), ('monarch', 0.3962482810020447), ('monarchy', 0.39430150389671326)]


In [8]:
print(word_vectors.most_similar("tower", topn=5))

[('towers', 0.8531750440597534), ('skyscraper', 0.6417425870895386), ('Tower', 0.639177143573761), ('spire', 0.594687819480896), ('responded_Understood_Atlasjet', 0.5931612253189087)]


In [10]:
print(word_vectors.similarity('woman', 'man'))

0.76640123


In [11]:
import numpy as np

In [12]:
vec_diff = word_vectors['man'] - word_vectors['women']
magnitude_of_difference = np.linalg.norm(vec_diff)

In [13]:
magnitude_of_difference

2.9112875

The closer the relation the lesser the magnitude of difference between the vectors (which is basically the distance between the vectors)